<a href="https://colab.research.google.com/github/UrmiDas1979/MLPractice/blob/main/LSTMpracticepro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install numpy pandas matplotlib scikit-learn tensorflow fastapi uvicorn pyngrok joblib

Upload Dataset

In [33]:
from google.colab import files
uploaded = files.upload()

Saving Stock.csv to Stock (1).csv


In [34]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

# Load data
data = pd.read_csv("Stock.csv")
close_prices = data['Close'].values.reshape(-1,1)

# Scale
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_prices)

# Create sequences
def create_sequences(data, window=60):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)

window = 60
X, y = create_sequences(scaled_data, window)

# Split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(window,1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save model
model.save("lstm_model.h5")
joblib.dump(scaler, "scaler.save")

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - loss: 0.2030
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0956
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0848
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0664
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0482
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0387
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0318
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0378
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0264
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0226


['scaler.save']

API code

In [35]:
!pip install fastapi uvicorn nest-asyncio

In [41]:
import nest_asyncio
import uvicorn
import threading
from fastapi import FastAPI, HTTPException
import numpy as np
from tensorflow.keras.models import load_model
import joblib

nest_asyncio.apply()

app = FastAPI()

# Load model safely (Keras 3 compatible)
model = load_model("/content/lstm_model.h5", compile=False)

# Load scaler
scaler = joblib.load("/content/scaler.save")

window = 60

@app.post("/predict")
def predict(data: dict):
    try:
        if "values" not in data:
            raise HTTPException(status_code=400, detail="Missing 'values' key")

        values = data["values"]

        if len(values) != window:
            raise HTTPException(
                status_code=400,
                detail=f"Input must contain exactly {window} values"
            )

        values = np.array(values).reshape(-1, 1)
        scaled = scaler.transform(values)
        scaled = scaled.reshape(1, window, 1)

        prediction = model.predict(scaled, verbose=0)
        predicted_price = scaler.inverse_transform(prediction)

        return {"predicted_price": float(predicted_price[0][0])}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# Run API in background thread
def run():
    uvicorn.run(app, host="127.0.0.1", port=5000)

thread = threading.Thread(target=run)
thread.start()

INFO:     Started server process [284]
INFO:     Waiting for application startup.


In [42]:
import requests
import time

time.sleep(3)  # give server time to start

data = {
    "values": [100.0]*60   # must match window size
}

response = requests.post(
    "http://127.0.0.1:5000/predict",
    json=data
)

print("Status Code:", response.status_code)
print("Response:", response.json())

INFO:     127.0.0.1:49196 - "POST /predict HTTP/1.1" 200 OK
Status Code: 200
Response: {'predicted_price': 152.81021118164062}
